<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex09-AlphaBeta_Alpha_Vantage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investments: Theory, Fundamental Analysis, and Data Driven Analytics**, Bates, Boyer, and Fletcher

# Example Chapter 9: Estimating Alpha Using Data from Alpha Vantage API  
  
In this example we estimate the alpha and beta of Johnson and Johnson (`ticker:JNJ`) relative to the market portfolio along with 95% confidence intervals. In this example, data for Johnson and Johnson returns come from the Alpha Vantage API. Data for the market and short-term T-bill returns come from [Ken French's website](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html). An introduction to the Alpha Vantage API is given in [EX07-Alpha_Vantage.ipynb](https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex07-Alpha_Vantage.ipynb).  

### Imports and Setup
We first import the `simple_finance`, `requests`, `pandas`, and `statsmodels` packages. To improve readability when printing a DataFrame to the consol, we then configure Pandas display options: (1) show all columns without truncation, (2) widen the output to 1000 characters so rows print on a single line, and (3) truncate long tables.


In [3]:
# Import the simple_finance.py package and avoid using a stale cached version.
!curl -s -O https://raw.githubusercontent.com/boyerb/Investments/master/functions/simple_finance.py
import importlib, simple_finance as sf
importlib.reload(sf)

import requests
import pandas as pd
import statsmodels.api as sm

pd.set_option('display.max_columns', None)   # Show all columns without truncation
pd.set_option('display.width', 1000)   # Set the display width so output stays on one line
pd.set_option("display.max_rows", 20) # Force truncation if DataFrame has more than 20 rows

### Load in Data for Johnson & Johnson Using the Alpha Vantage API
Replace `your_key` in the script with your actual API key. If you don't supply your own API key, the code will still run, but the number of daily requests will be **limited by IP address**.   We then create returns as a percent change in the adjusted close to account for splits, dividends, and other corporate events.

In [4]:
# Replace the text 'your_key' with your own APPI key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol=JNJ&apikey=your_key'

# Send an HTTP GET request to the API endpoint
r = requests.get(url)
df = sf.parse_monthly_alpha_vantage_response(r)

# Create returns column using Adjusted Close
df["Return"] = df["Adjusted Close"].pct_change()
df = df.dropna()  # drop row with missing value
print(df)

            Open    High     Low   Close  Adjusted Close       Volume  Dividend Amount    Return
2000-01   93.130   96.94   80.50   86.06         22.0597   76758100.0             0.00 -0.077104
2000-02   85.440   86.31   70.06   72.00         18.5224   93261000.0             0.28 -0.160351
2000-03   72.250   82.25   66.13   70.25         18.0722  139902700.0             0.00 -0.024306
2000-04   70.630   84.44   70.00   82.50         21.2236   85393700.0             0.00  0.174378
2000-05   82.250   90.00   81.88   89.50         23.1074   60104100.0             0.32  0.088760
...          ...     ...     ...     ...             ...          ...              ...       ...
2025-06  154.480  157.67  149.04  152.75        151.6331  159006719.0             0.00 -0.015849
2025-07  153.000  169.99  152.81  164.74        163.5354  187622833.0             0.00  0.078494
2025-08  165.475  181.16  164.79  177.17        177.1700  174938970.0             1.30  0.083374
2025-09  177.470  185.99  173.

### Load Returns for the Market and Short-Term T-Bills
This function retrieves monthly data for the Fama–French Five-Factor model from [Ken French's website](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html). From the resulting DataFrame, we will extract the market’s excess return `Mkt-RF` and the return on short-term T-bills `RF`.


In [5]:
FF = sf.get_ff5(start_date="2010-01", end_date="2024-12")
print(FF)

         Mkt-RF     SMB     HML     RMW     CMA      RF
date                                                   
2010-01 -0.0335  0.0040  0.0033 -0.0108  0.0051  0.0000
2010-02  0.0339  0.0149  0.0318 -0.0029  0.0142  0.0000
2010-03  0.0630  0.0183  0.0219 -0.0061  0.0174  0.0001
2010-04  0.0200  0.0496  0.0296  0.0061  0.0175  0.0001
2010-05 -0.0790  0.0008 -0.0248  0.0130 -0.0024  0.0001
...         ...     ...     ...     ...     ...     ...
2024-08  0.0161 -0.0355 -0.0110  0.0075  0.0082  0.0048
2024-09  0.0173 -0.0092 -0.0277  0.0018 -0.0029  0.0040
2024-10 -0.0100 -0.0088  0.0086 -0.0142  0.0098  0.0039
2024-11  0.0649  0.0460  0.0015 -0.0230 -0.0205  0.0040
2024-12 -0.0315 -0.0383 -0.0300  0.0190 -0.0121  0.0037

[180 rows x 6 columns]


### Merge Two Data Sources
In this code we merge the data from Alpha Vantage and Ken French's website together. We then create the excess return for Johnson & Johnson.

In [ ]:
merged = df.merge(FF, left_index=True, right_index=True, how="inner")
merged = merged[["Return", "Mkt-RF", "RF"]]
merged["Ret-RF"] = merged["Return"] - merged["RF"]
print(merged)

### Estimate Alpha, Beta, and 95% Confidence Intervals
Here we estimate the alpha and beta of Johnson & Johnson relative to the market portfolio along with the 95% confidence intervals.

In [ ]:
X=merged['Mkt-RF']
Y=merged['Ret-RF']
X=sm.add_constant(X)  # we specify that we want to add a constant to the regression equation
model=sm.OLS(Y,X).fit() # run the regression
params = model.params # parameter values
conf_int = model.conf_int(alpha=0.05)  # 95% CIs

# create a table of output
results_df = pd.DataFrame({
    'Parameter': params.index, # 1st column: parameter name
    'Estimate': params.values, # 2nd column: parameer value
    'CI Lower': conf_int[0].values, # 3rd column: lower bound on 95\% CI
    'CI Upper': conf_int[1].values  # 4th column: upper bound on 95\% CI
})
print(results_df)